In [1]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
import pickle

In [3]:
from tensorflow.keras.preprocessing import image

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score,log_loss, confusion_matrix, roc_curve, ConfusionMatrixDisplay, RocCurveDisplay

In [5]:
import mlflow

In [6]:
def create_experiment(experiment_name,run_name,model,mloss,maccuracy):
    mlflow.set_tracking_uri("http://localhost:5000") # Descomentar esta línea si desea usar cualquier base de datos como sqlite como almacenamiento de back-end
    #mlflow.set_tracking_uri("http://localhost:8080")
    mlflow.set_experiment(experiment_name)

    with mlflow.start_run(run_name=run_name):
        mlflow.sklearn.log_model(model, "model")
        mlflow.set_tag("tag1", "vgg161")
        mlflow.log_metric('loss', mloss)
        mlflow.log_metric('accuracy', maccuracy)
    print('La ejecución: %s fue registrada en el experimento: %s' %(run_name, experiment_name))

In [7]:
data_train=pd.read_csv('train.csv')

In [8]:
data_train

,file,age,gender,race,service_test
0,train/1.jpg,50-59,Male,East Asian,True
1,train/2.jpg,30-39,Female,Indian,False
2,train/3.jpg,3-9,Female,Black,False
3,train/4.jpg,20-29,Female,Indian,True
4,train/5.jpg,20-29,Female,Indian,True
...,...,...,...,...,...
86739,train/86740.jpg,20-29,Male,Indian,True
86740,train/86741.jpg,10-19,Male,Indian,True
86741,train/86742.jpg,more than 70,Female,Indian,True
86742,train/86743.jpg,10-19,Female,Black,True


In [9]:
data_test=pd.read_csv('val.csv')

In [10]:
data_test

,file,age,gender,race,service_test
0,val/1.jpg,3-9,Male,East Asian,False
1,val/2.jpg,50-59,Female,East Asian,True
2,val/3.jpg,30-39,Male,White,True
3,val/4.jpg,20-29,Female,Latino_Hispanic,True
4,val/5.jpg,20-29,Male,Southeast Asian,False
...,...,...,...,...,...
10949,val/10950.jpg,30-39,Male,White,True
10950,val/10951.jpg,50-59,Male,White,False
10951,val/10952.jpg,60-69,Male,Latino_Hispanic,False
10952,val/10953.jpg,20-29,Female,East Asian,False


In [11]:
data_train1=data_train[["file","race"]]

In [12]:
data_train1

,file,race
0,train/1.jpg,East Asian
1,train/2.jpg,Indian
2,train/3.jpg,Black
3,train/4.jpg,Indian
4,train/5.jpg,Indian
...,...,...
86739,train/86740.jpg,Indian
86740,train/86741.jpg,Indian
86741,train/86742.jpg,Indian
86742,train/86743.jpg,Black


In [13]:
data_train1.columns=["id","label"]

In [14]:
data_train1

,id,label
0,train/1.jpg,East Asian
1,train/2.jpg,Indian
2,train/3.jpg,Black
3,train/4.jpg,Indian
4,train/5.jpg,Indian
...,...,...
86739,train/86740.jpg,Indian
86740,train/86741.jpg,Indian
86741,train/86742.jpg,Indian
86742,train/86743.jpg,Black


In [15]:
data_test1=data_test[["file","race"]]

In [16]:
data_test1

,file,race
0,val/1.jpg,East Asian
1,val/2.jpg,East Asian
2,val/3.jpg,White
3,val/4.jpg,Latino_Hispanic
4,val/5.jpg,Southeast Asian
...,...,...
10949,val/10950.jpg,White
10950,val/10951.jpg,White
10951,val/10952.jpg,Latino_Hispanic
10952,val/10953.jpg,East Asian


In [17]:
data_test1.columns=["id","label"]

In [18]:
data_test1

,id,label
0,val/1.jpg,East Asian
1,val/2.jpg,East Asian
2,val/3.jpg,White
3,val/4.jpg,Latino_Hispanic
4,val/5.jpg,Southeast Asian
...,...,...
10949,val/10950.jpg,White
10950,val/10951.jpg,White
10951,val/10952.jpg,Latino_Hispanic
10952,val/10953.jpg,East Asian


In [19]:
data_train2=data_train1.head(50)

In [20]:
data_train2

,id,label
0,train/1.jpg,East Asian
1,train/2.jpg,Indian
2,train/3.jpg,Black
3,train/4.jpg,Indian
4,train/5.jpg,Indian
5,train/6.jpg,White
6,train/7.jpg,Middle Eastern
7,train/8.jpg,Indian
8,train/9.jpg,White
9,train/10.jpg,Middle Eastern


In [21]:
data_test2=data_test1.head(50)

In [22]:
data_test2

,id,label
0,val/1.jpg,East Asian
1,val/2.jpg,East Asian
2,val/3.jpg,White
3,val/4.jpg,Latino_Hispanic
4,val/5.jpg,Southeast Asian
5,val/6.jpg,Latino_Hispanic
6,val/7.jpg,Black
7,val/8.jpg,East Asian
8,val/9.jpg,Southeast Asian
9,val/10.jpg,Southeast Asian


In [23]:
batch_size = 32
img_height = 224
img_width = 224

In [24]:
data_dirTrain = ''
data_dirTest = ''

In [25]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1 / 255., horizontal_flip=True)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1 / 255.)

In [29]:
train_ds = train_datagen.flow_from_dataframe(
  data_train2,
  directory=data_dirTrain,
  x_col='id',
  y_col='label',
  seed=42,
  shuffle=True,
  target_size=(img_height, img_width),
  class_mode="categorical",
  batch_size=batch_size)

Found 50 validated image filenames belonging to 7 classes.


In [30]:
val_ds = val_datagen.flow_from_dataframe(
  data_test2,
  directory=data_dirTest,
  x_col='id',
  y_col='label',
  target_size=(img_height, img_width),
  class_mode="categorical",
  batch_size=batch_size)

Found 50 validated image filenames belonging to 7 classes.


In [31]:
class_to_idx = train_ds.class_indices
class_to_idx

{'Black': 0,
 'East Asian': 1,
 'Indian': 2,
 'Latino_Hispanic': 3,
 'Middle Eastern': 4,
 'Southeast Asian': 5,
 'White': 6}

In [32]:
idx_to_class_list = [(value, key) for value, key in class_to_idx.items()]
idx_to_class = {}
for element in idx_to_class_list:
    idx_to_class[element[1]] = element[0]

idx_to_class

{0: 'Black',
 1: 'East Asian',
 2: 'Indian',
 3: 'Latino_Hispanic',
 4: 'Middle Eastern',
 5: 'Southeast Asian',
 6: 'White'}

In [33]:
num_classes = len(idx_to_class)

In [34]:
num_classes

7

Modelo armado desde cero

In [38]:


modelMio = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(128, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [39]:
modelMio.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
modelMio.fit(train_ds,
          validation_data=val_ds,
          epochs=2)

Epoch 1/2


2/2 [==============================] - 122s 120s/step - loss: 2.0173 - accuracy: 0.1600 - val_loss: 2.0168 - val_accuracy: 0.1397
Epoch 2/2
2/2 [==============================] - 81s 81s/step - loss: 1.9942 - accuracy: 0.1800 - val_loss: 1.9663 - val_accuracy: 0.2121


In [41]:
input_shape=(img_height, img_width, 3)

In [42]:
imagen1=data_test1["id"][10]

In [43]:
imagen1

'val/11.jpg'

In [49]:
new_img = image.load_img(imagen1, target_size=input_shape[:2])

In [50]:
new_img = image.img_to_array(new_img)
new_img = np.expand_dims(new_img, axis=0)
new_img = new_img / 255.0

In [51]:
prediction = modelMio.predict(new_img)

1/1 [==============================] - 0s 184ms/step


In [52]:
laPrediccion=np.argmax(prediction)

In [53]:
lasCategoriasArray1=pd.DataFrame.from_dict(idx_to_class, orient="index")

In [54]:
laPrediccionLiteral=lasCategoriasArray1.iloc[laPrediccion,0]

In [55]:
print(f'La prediccion es {laPrediccionLiteral}')

La prediccion es White


In [65]:
loss,accuracy=modelMio.evaluate(val_ds)

2/2 [==============================] - 2s 645ms/step - loss: 2.0261 - accuracy: 0.1600


In [67]:
print(f'Loss:{loss}, accuracy {accuracy}')

Loss:2.0261051654815674, accuracy 0.1599999964237213


In [68]:
loss

2.0261051654815674

In [82]:
accuracy

0.1599999964237213

In [83]:
experiment_name = "experimento_mio1"
run_name = "runmio_1"

create_experiment(experiment_name, 
                  run_name,modelMio,
                  loss,accuracy
                 )

2024/02/14 17:00:28 INFO mlflow.tracking.fluent: Experiment with name 'experimento_mio1' does not exist. Creating a new experiment.
2024/02/14 17:01:07 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\win10\AppData\Local\Temp\tmpxqfc84aq\model\model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.4.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback.


La ejecución: runmio_1 fue registrada en el experimento: experimento_mio1


Modelo en base a vgg16

In [35]:
vgg16 = tf.keras.applications.VGG16(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')

vgg16.trainable = False

modelvgg16 = tf.keras.Sequential([
  vgg16,
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [36]:
modelvgg16.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
modelvgg16.fit(train_ds,
          validation_data=val_ds,
          epochs=2)

Epoch 1/2


2/2 [==============================] - 136s 95s/step - loss: 2.4409 - accuracy: 0.1600 - val_loss: 2.1261 - val_accuracy: 0.1200
Epoch 2/2
2/2 [==============================] - 84s 70s/step - loss: 2.1646 - accuracy: 0.2000 - val_loss: 2.2638 - val_accuracy: 0.1200


In [38]:
input_shape=(img_height, img_width, 3)

In [117]:
imagen1=data_test1["id"][450]

In [118]:
imagen1

'val/451.jpg'

In [119]:
# Cargar y preprocesar una nueva imagen (reemplazar "ruta/de/imagen.jpg" con la ruta de la imagen)
from tensorflow.keras.preprocessing import image

#new_img = image.load_img("/content/0001.jpg", target_size=input_shape[:2])
new_img = image.load_img(imagen1, target_size=input_shape[:2])



In [120]:
new_img = image.img_to_array(new_img)
new_img = np.expand_dims(new_img, axis=0)
new_img = new_img / 255.0


In [121]:
prediction = modelvgg16.predict(new_img)

1/1 [==============================] - 0s 373ms/step


In [122]:
prediction

array([[0.07978797, 0.07270153, 0.2128977 , 0.08025935, 0.16005337,
        0.09521378, 0.2990863 ]], dtype=float32)

In [123]:
laPrediccion=np.argmax(prediction)
laPrediccion

6

In [47]:
idx_to_class

{0: 'Black',
 1: 'East Asian',
 2: 'Indian',
 3: 'Latino_Hispanic',
 4: 'Middle Eastern',
 5: 'Southeast Asian',
 6: 'White'}

In [48]:
type(idx_to_class)

dict

In [49]:
list(idx_to_class)

[0, 1, 2, 3, 4, 5, 6]

In [50]:
lasCategoriasArray=np.array(list(idx_to_class))

In [51]:
lasCategoriasArray

array([0, 1, 2, 3, 4, 5, 6])

In [52]:
lasCategoriasArray=np.array(idx_to_class)

In [53]:
lasCategoriasArray

array({0: 'Black', 1: 'East Asian', 2: 'Indian', 3: 'Latino_Hispanic', 4: 'Middle Eastern', 5: 'Southeast Asian', 6: 'White'},
      dtype=object)

In [54]:
lasCategoriasArray1=pd.DataFrame.from_dict(idx_to_class, orient="index")

In [55]:
lasCategoriasArray1

,0
0,Black
1,East Asian
2,Indian
3,Latino_Hispanic
4,Middle Eastern
5,Southeast Asian
6,White


In [56]:
lasCategoriasArray1.iloc[0,0]

'Black'

In [71]:
laPrediccionLiteral=lasCategoriasArray1.iloc[laPrediccion,0]

In [72]:
laPrediccionLiteral

'Indian'

In [73]:
print(f'La prediccion es {laPrediccionLiteral}')

La prediccion es Indian


In [60]:
loss,accuracy=modelvgg16.evaluate(val_ds)

2/2 [==============================] - 1965s 22s/step - loss: 2.2638 - accuracy: 0.1200


In [62]:
experiment_name = "experimento_vgg161"
run_name = "runvgg16_1"

create_experiment(experiment_name, 
                  run_name,modelvgg16,
                  loss,accuracy
                 )

2024/02/15 19:04:29 INFO mlflow.tracking.fluent: Experiment with name 'experimento_vgg161' does not exist. Creating a new experiment.
C:\ProgramData\Anaconda3\envs\tf1\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


La ejecución: runvgg16_1 fue registrada en el experimento: experimento_vgg161


In [87]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [90]:
filename = 'modelMio.pkl' 

In [89]:
modelMio.save('modelMio.h5')

C:\ProgramData\Anaconda3\envs\tf1\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [93]:
pickle.dump(modelMio, open(filename,'wb'))

In [94]:
modelMio.save('modelMio.h5')

In [96]:
modelMio.save('pb1')

INFO:tensorflow:Assets written to: pb1\assets


INFO:tensorflow:Assets written to: pb1\assets


In [86]:
!python --version

Python 3.9.18


In [63]:
modelvgg16.save('pb2')

INFO:tensorflow:Assets written to: pb2\assets


INFO:tensorflow:Assets written to: pb2\assets


In [126]:
races=['Black','East Asian','Indian','Latino_Hispanic','Middle Eastern','Southeast Asian','White']

In [127]:
races[laPrediccion]

'White'